# eval

In [ ]:
import numpy as np
from src.utils import get_args_and_config
from src.data import get_data
from src.eval import corr
from src.model import forward
from src.fmri import plot_brain

In [ ]:
args, config = get_args_and_config()
data = get_data(args, config)

In [ ]:
for subject, (fold, test) in data.items():
    lh_params = np.load(f"results/models/{subject}_lh_best_model.npy", allow_pickle=True).item()
    rh_params = np.load(f"results/models/{subject}_rh_best_model.npy", allow_pickle=True).item()
    lh_corr = corr(forward(lh_params, test[0])[0], test[1])
    rh_corr = corr(forward(rh_params, test[0])[1], test[2])
    break

In [ ]:
plot_brain(lh_corr, subject, "lh")

In [ ]:
plot_brain(rh_corr, subject, "rh")